In [1]:
import numpy as np
import scipy.io

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats as st
from collections import Counter
from sklearn.decomposition import PCA

# 1. Implement Decision Tree method (50 points)

In [2]:
class Node:
    def __init__(self, feature=None, criteria=None, left=None, right=None, value=None):
        self.feature=feature
        self.criteria = criteria
        self.left = left
        self.right = right
        self.value = value
    def is_leaf_node(self):
        return self.value is not None
        
class Decision_Tree:
    def __init__(self, min_sample_split=2, max_depth=100, n_feature= None):
        self.min_sample_split=min_sample_split
        self.max_depth=max_depth
        self.n_feature=n_feature
        self.root=None
    
    
    def fit(self, train_x, train_y):
        if not self.n_feature:
            self.n_feature=train_x.shape[1]
        else:
            self.n_feature=min(train_x.shape[1],self.n_feature)
        self.root=self.grow(train_x, train_y)
        
    
    
    def grow(self, train_x, train_y, depth=0):
        n, m = train_x.shape
        n_class = len(np.unique(train_y))
        
        # check stoppping criteria
        if depth > self.max_depth or n<self.min_sample_split or n_class ==1:
            
            #counter = Counter(train_y.to_list())
            
            #val = counter.most_common(1)[0][0] 
            #print(train_y)
            leaf_val = st.mode(train_y.reshape(-1).tolist())[0][0]
            #leaf_val=0
            return Node(value=leaf_val)
        feature_idx = np.random.choice(m, self.n_feature, replace = False)
        
        # find the best split
        best_feat, best_thre= self.best_split(train_x, train_y, feature_idx)[:-1]
        
        # create child node
        left_idx = np.argwhere(train_x[:, best_feat]<= best_thre).flatten()
        right_idx = np.argwhere(train_x[:, best_feat]> best_thre).flatten()
        left = self.grow(train_x[left_idx,:], train_y[left_idx], depth+1)
        right = self.grow(train_x[right_idx,:], train_y[right_idx], depth+1)
        return Node(best_feat, best_thre, left, right)
        
    def entropy(self, y):
        _,counts = np.unique(y, return_counts=True)
        #print('counts:',counts)
        prob = counts/len(y)
        return -np.sum(prob*np.log2(prob))
    
    def best_split(self, train_x, train_y, feature_idx):
        min_en_list=[]
        best=-np.inf
        split_idx=None
        split_criteria = None
        for idx in feature_idx:
            min_en = np.inf
            xi = train_x[:,idx]
            options = np.sort(np.unique(xi))
            #print(options.shape)
            for opt in options:
                #print(xi, train_y,opt)
                ig, en = self.information_gain(xi, train_y, opt)
                #print('opt:',opt,'\n', ig, '\n',en)
                
                if en < min_en:
                    min_en = en
                if ig > best:
                    best = ig
                    split_idx = idx
                    split_criteria = opt
            min_en_list.append(min_en)
               
        return split_idx, split_criteria, min_en_list
        
    def information_gain(self, xi, train_y, opt):
        # Calculate parents entropy
        parent_entropy = self.entropy(train_y)
        
        # Calculate weights*childrent entropy
        left_idx = np.argwhere(xi<=opt).flatten()
        #print('left_idx',left_idx)
        right_idx = np.argwhere(xi>opt).flatten()
        #print('right_idx',right_idx)
        if len(left_idx)==0 or len(right_idx)==0:
            
            return 0, np.inf
        
        n = len(train_y)
        nl, nr = len(left_idx), len(right_idx)
        total = nl+nr
        left_entropy = self.entropy(train_y[left_idx])
        right_entropy = self.entropy(train_y[right_idx])
        #print('left_e:',left_entropy)
        #print('right_e:',right_entropy)
        children_entropy = (nl/total)*left_entropy + (nr/total)*right_entropy
        ig = parent_entropy-children_entropy
        
        return ig, children_entropy
        
    
    def predict(self, test_x):
        res=[]
        for x in test_x:
            res.append(self.traverse(x, self.root))
        return np.array(res)
        
    def traverse(self, x, node):
        if node.is_leaf_node():
            return node.value
        if x[node.feature]<=node.criteria:
            return self.traverse(x, node.left)
        return self.traverse(x, node.right)

    

* Ref:  
1. https://youtu.be/NxEHSAfFlK8  
2. https://anderfernandez.com/en/blog/code-decision-tree-python-from-scratch/  
3. https://towardsdatascience.com/entropy-how-decision-trees-make-decisions-2946b9c18c8

# 2. Load train and test mat files, perform Decision Tree and report acuracy on the test dataset (20 points)

In [3]:
train = scipy.io.loadmat('train.mat')
test = scipy.io.loadmat('test.mat')

In [4]:
train_x = train['features']
train_y = train['labels']
test_x = test['features']
test_y = test['labels']

In [5]:
def accuracy(pred, test_y):
    return np.sum(pred==test_y)/len(test_y)

### Fit all the features

In [6]:
model = Decision_Tree()
model.fit(train_x, train_y)
pred = model.predict(test_x)

In [7]:
print('accuracy:',round(accuracy(pred, test_y),2),'%')

accuracy: 96.16 %


### Fit the 200 features that has the lowest entropy

In [8]:
model = Decision_Tree()

In [9]:
min_en_list = model.best_split(train_x, train_y, [i for i in range(train_x.shape[1])])[-1]

In [10]:
# select 200 lowest entropy to be the features
low_en_idx = np.array(min_en_list).argsort()[:200]
model.fit(train_x[:,low_en_idx], train_y)
pred = model.predict(test_x)
print('accuracy:',round(accuracy(pred, test_y),2),'%')

accuracy: 97.07 %


# 3. Reduce the dimensionality of features using PCA to low dimensions (e.g., 10, 3 ) and report the accuracy of the test datasets (20 points)

In [11]:
for i in [3,10,20,40,60,100,200]:
    pca = PCA(n_components=i)
    reduced_train = pca.fit_transform(train_x)
    #print(reduced_train.shape)
    reduced_test = pca.fit_transform(test_x)
    model.fit(reduced_train, train_y)
    pred = model.predict(reduced_test)
    print(f'accuracy in {i} dimensions:',round(accuracy(pred, test_y),2),'%')

accuracy in 3 dimensions: 95.86 %
accuracy in 10 dimensions: 96.13 %
accuracy in 20 dimensions: 95.03 %
accuracy in 40 dimensions: 95.38 %
accuracy in 60 dimensions: 95.7 %
accuracy in 100 dimensions: 95.75 %
accuracy in 200 dimensions: 95.7 %


# 4. Compare results with your KNN model (10 points)

The selected features based on entropy has the highest predicted accuracy. 